# Simple GPROF rain rate retrieval

In this notebook we will apply QRNNs to retrieve rain rates from passive microwave observations from the Global Precipitation Measurement (GPM) mission. We will then use the model to classify pixels based on their rain rate and compare the performance of the classifier to that of the current algorithm. 

In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
from quantnn.examples.gprof_simple import GPROFDataset, download_data
download_data()

## The Data

The input data consists of single-pixel observations from the GMI radiometer combined with the surface type, column-integrated water vapor and two-meter temperature. Expanding the 14 surface types to one-hot encoding results in 29 input features.

The output data consists of surface rain rates, which are, for the largest part, derived from GMI observations combined with the precipitation radar, which is flown together with GMI on board the GPM Core Observatory satellite. The combined radar-radiometer observations improve the accuracy of the rain retrieval, which is why this data is suitable to be used as ground truth.

In [2]:
training_data = GPROFDataset("data/training_data_gmi_small.nc", batch_size=128)
validation_data = GPROFDataset("data/training_data_gmi_small.nc", batch_size=128, normalizer=training_data.normalizer)

In [3]:
from torch.utils.data import DataLoader
training_loader = DataLoader(training_data, batch_size=None, num_workers=2)
validation_loader = DataLoader(validation_data, batch_size=None, num_workers=2)

## Defining a QRNN model

In [4]:
import quantnn as qn
quantiles = [0.01, 0.05, 0.15, 0.25, 0.35, 0.45, 0.5, 0.55, 0.65, 0.75, 0.85, 0.95, 0.99]
qrnn = qn.QRNN(n_inputs=29, quantiles=quantiles, model=(4, 256, "relu"))

## Training the QRNN

In [5]:
import torch
n_epochs = 10
optimizer = torch.optim.SGD(qrnn.model.parameters(), lr=0.1, momentum=0.9)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, n_epochs)
qrnn.train(training_loader,
           validation_loader,
           optimizer=optimizer,
           scheduler=scheduler,
           n_epochs=n_epochs,
           device="gpu")
optimizer = torch.optim.SGD(qrnn.model.parameters(), lr=0.01, momentum=0.9)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, n_epochs)
qrnn.train(training_loader,
           validation_loader,
           optimizer=optimizer,
           scheduler=scheduler,
           n_epochs=n_epochs,
           device="gpu")
optimizer = torch.optim.SGD(qrnn.model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, n_epochs)
qrnn.train(training_loader,
           validation_loader,
           optimizer=optimizer,
           scheduler=scheduler,
           n_epochs=n_epochs,
           device="gpu")

Epoch  1 / 10: train. loss = 0.0197, val. loss = 0.0187, lr. = 0.1000, time = 59.849901 s
Epoch  2 / 10: train. loss = 0.0175, val. loss = 0.0181, lr. = 0.0976, time = 57.874561 s
Epoch  3 / 10: train. loss = 0.0169, val. loss = 0.0177, lr. = 0.0905, time = 58.140045 s
Epoch  4 / 10: train. loss = 0.0164, val. loss = 0.0174, lr. = 0.0794, time = 58.50001 s
Epoch  5 / 10: train. loss = 0.0160, val. loss = 0.0171, lr. = 0.0655, time = 57.944518 s
Epoch  6 / 10: train. loss = 0.0156, val. loss = 0.0169, lr. = 0.0500, time = 57.821666 s
Epoch  7 / 10: train. loss = 0.0152, val. loss = 0.0166, lr. = 0.0345, time = 58.348087 s
Epoch  8 / 10: train. loss = 0.0149, val. loss = 0.0164, lr. = 0.0206, time = 57.638794 s
Epoch  9 / 10: train. loss = 0.0146, val. loss = 0.0162, lr. = 0.0095, time = 58.148179 s
Epoch 10 / 10: train. loss = 0.0144, val. loss = 0.0160, lr. = 0.0024, time = 57.940222 s
Epoch  1 / 10: train. loss = 0.0145, val. loss = 0.0144, lr. = 0.0100, time = 58.036201 s
Epoch  2 / 

{'training_errors': [0.019676853695162495,
  0.017488023740689806,
  0.016883232884681725,
  0.016414961717262377,
  0.015973747502462596,
  0.015567858074627851,
  0.01519048616364194,
  0.014854037944624228,
  0.01457825569161006,
  0.01440717304811162,
  0.014535996318773697,
  0.014505150770559956,
  0.014460944863172202,
  0.014411092704782448,
  0.014358306339428162,
  0.014303201402018158,
  0.014252274930432383,
  0.014211172438434236,
  0.014182657265558528,
  0.014166630544527105,
  0.014162283242236633,
  0.014161297585052572,
  0.01416039977502488,
  0.01415959142190979,
  0.014158872159898911,
  0.014158237466472086,
  0.014157720947930672,
  0.01415730519433801,
  0.01415693029079439,
  0.01415666496827965],
 'validation_errors': [0.018707160967971107,
  0.017395331839085607,
  0.017124375757569213,
  0.01655358172949176,
  0.015963830074235914,
  0.01541334416074616,
  0.014981719197775033,
  0.014756156523286287,
  0.01448327758246876,
  0.01438092937800302,
  0.0144416

## Classifying raining pixels

We will test our retrieval by using it to predict whether the rain rate at a given pixel is larger than a threshold of $0.01\ \text{mm/h}$. The estimated probability of a given pixel being larger than this value is also part of the GPROF retrieval output and we will use this output to evaluate the QRNN retrieval.

In [6]:
import quantnn as q
from quantnn.examples.gprof_rr import GPROFTestset
test_data = GPROFTestset("data/test_data_gmi_small.nc", normalizer=training_data.normalizer)
p_qrnn = qrnn.probability_larger_than(test_data.x, 0.01).detach().numpy()
p_gprof = test_data.y_pop / 100.0
truth = test_data.y.ravel() > 0.01

ModuleNotFoundError: No module named 'quantnn.examples.gprof_rr'

In [ ]:
def precision_and_recall_curve(p_pred, truth, n=101):
    ps = np.linspace(0, 1, n)
    precision = np.zeros(n)
    recall = np.zeros(n)
    for i, p in enumerate(ps):
        predicted = p_pred > p
        precision[i] = np.sum(predicted * truth) / np.sum(predicted)
        recall[i] = np.sum(predicted * truth) / np.sum(truth)
    return precision, recall

precision_qrnn, recall_qrnn = precision_and_recall_curve(p_qrnn, truth)
precision_gprof, recall_gprof = precision_and_recall_curve(p_gprof, truth)

In [ ]:
from quantnn.plotting import set_style
set_style()
f, ax = plt.subplots(1, 1)
ax.plot(recall_qrnn, precision_qrnn, label="QRNN")
ax.plot(recall_gprof, precision_gprof, label="GPROF")
ax.set_xlabel("Recall")
ax.set_ylabel("Precision")
ax.legend()